In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains 

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from PIL import Image
from io import BytesIO
import time
import csv
import os
import base64
import concurrent.futures
import random


In [3]:
import selenium
print("versión selenium:", selenium.__version__)

versión selenium: 4.19.0


In [4]:
# Nombre del archivo CSV
denomination_file = "-actas-vigentes.csv"
denomination_images = "-images"
# Ruta completa al archivo CSV en el directorio "data" dentro del directorio actual
initial_data = ["NRO ACTA", 
                "TITULAR", 
                "FECHA INGRESO",
                "CLASE",
                "DENOMINACION",
                "TIPO DE MARCA",
                "NRO RESOLUCION",
                "ESTADO",
                "VENCIMIENTO",
                "PROTECCION",
                "LIMITACION",
                "NOMBRE IMAGEN"]

def routeFile(number):
    name = "class" + str(number) + denomination_file
    return os.path.join(os.getcwd(), "data", name)

def routeFolderImages(number):
    name = "class" + str(number) + denomination_images
    return os.path.join(os.getcwd(), "data", name)

def createFile(number):
    with open(routeFile(number), mode='w', newline='') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerow(initial_data)

def appendLines(number, data):
    with open(routeFile(number), mode='a', newline='') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
                
        for line in data:
            escritor_csv.writerow(line)

def storeImage(number, url, act_number):
    route = routeFolderImages(number)
    base64_image = url.split(",")[1]
    image_bytes = base64.b64decode(base64_image)
    image = Image.open(BytesIO(image_bytes))
    if not os.path.exists(route):
        os.makedirs(route)
    image.save(os.path.join(route, act_number + ".jpg"))

In [10]:
def botClass(classNumber):
    # Functions
    def makeCell(line):
        aLine = []
        for cell in line:
            aLine.append(cell.text)
        aLine.pop()
        return aLine

    def wait_for_element_to_hide(driver, locator, timeout=10):
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((locator))
        )
    
    def CreateChromeDriver(classNumber):
        try:
            options = Options()
            options.add_argument('--start-maximized')
            options.add_argument('--disable-extensions')

            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
            driver.get('https://portaltramites.inpi.gob.ar/marcasconsultas/busqueda/?Cod_Funcion=NQA0ADEA')
            return driver, False
        except Exception as e:
            print("Error while creating driver for class: ", classNumber)
            print(e)
            time.sleep(5)
            return None, True
    
    def MakeSearch(driver, classNumber):
        try:
            collapsables = driver.find_elements(By.CLASS_NAME, "accordion-toggle");
            desiredText = "BUSCADOR"
            for collapsable in collapsables:
                if desiredText in collapsable.text:
                    collapsable.click()
                
            time.sleep(2)
            class_dropdown = Select(driver.find_element(By.ID, "clase"));
            class_dropdown.select_by_value(str(classNumber))

            checkbox = driver.find_element(By.CLASS_NAME, "glyphicon-ok")
            checkbox.click()
            time.sleep(1)
            search_button = driver.find_element(By.ID, "BtnBuscarAvanzada")
            search_button.click()
            return driver, False
        except Exception as e:
            print("Error while making search for class: ", classNumber)
            print(e)
            time.sleep(5)
            return driver, True

    def GoToDetailPage(driver, classNumber):
            try:
                table = driver.find_element(By.ID, "tblGrillaMarcas")
                tbody = table.find_element(By.TAG_NAME, 'tbody')
                tableElements = tbody.find_elements(By.TAG_NAME, "tr")
                detailPage = tableElements[n].find_element(By.CLASS_NAME, "fa-plus-square")

                line = makeCell(tableElements[n].find_elements(By.TAG_NAME, "td"))
                hover = ActionChains(driver).move_to_element(detailPage)
                hover.perform()

                detailPage.click() 
                return driver, line, False
            except Exception as e:
                print("Error while making search for class: ", classNumber)
                print(e)
                time.sleep(5)
                return driver, [], True
    # First search
    print("Building driver for class ", classNumber)
    hasDriverError = True
    while hasDriverError:
        driver, hasDriverError = CreateChromeDriver(classNumber)

    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "BtnBuscarAvanzada"))
    )
    time.sleep(5)

    print("generating search for class ", classNumber)
    hasSearchError = True
    while hasSearchError:
        driver, hasSearchError = MakeSearch(driver, classNumber)

    createFile(classNumber)
    print("Created folder for class", classNumber)
    
    print("Go to table", classNumber)
    wait_for_element_to_hide(driver, (By.CSS_SELECTOR, ".fixed-table-loading"), timeout=300)
    time.sleep(5)

    print("loading first entry for class", classNumber)
    tab_original = driver.current_window_handle
    last_page_number = driver.find_element(By.CLASS_NAME, "page-last").text

    for i in range(1, int(last_page_number)):
        for n in range(0,10):

            line = [".",".",".",".",".",".",".",".","."]
            hasDetailPageError = True
            while hasDetailPageError:
                driver, line ,hasDetailPageError = GoToDetailPage(driver, classNumber)
                if hasDetailPageError:
                    driver.switch_to.window(tab_original)
                    time.sleep(10)
            p = driver.current_window_handle
            chwd = driver.window_handles
            for w in chwd:
                if(w!=p):
                    driver.switch_to.window(w)

            print("Waiting for protection, limitation in class: " + str(classNumber))
            WebDriverWait(driver, 100).until(
                EC.presence_of_element_located((By.ID, "collapse-two"))
            )
            time.sleep(3)
            data = driver.find_element(By.ID, "collapse-two")
            labels = data.find_elements(By.TAG_NAME, "label")
            for label in labels:
                if "PROTECCION" in label.text:
                    try:
                        line.append(label.find_element(By.TAG_NAME, "span").text)
                    except:
                        line.append("")
                if "LIMITACION" in label.text:
                    try:
                        limitation = label.text.split(": ")[1]
                        line.append(limitation[0:200])
                    except:
                        line.append("")
            try:
                imageId = driver.find_element(By.ID, "logo")  
                image = imageId.find_element(By.TAG_NAME, "img")
                image_url = image.get_attribute("src")
                storeImage(number=classNumber, url=image_url, act_number=line[0])
                line.append(line[0] + ".jpg")
            except:
                line.append("")
            appendLines(number=classNumber, data=[line])
            driver.close()
            time.sleep(2)
            driver.switch_to.window(tab_original)
        print("I finished page: " + str(i) + " for class: ", str(classNumber))
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.CLASS_NAME, "page-next"))
        )
        next = driver.find_element(By.CLASS_NAME, "page-next")
        clickeablePage = next.find_element(By.TAG_NAME, "a")
        clickeablePage.click()
        wait_for_element_to_hide(driver, (By.CSS_SELECTOR, ".fixed-table-loading"), timeout=300)
        time.sleep(2)
        driver.switch_to.window(tab_original)
    driver.close()

botClass(2)

Building driver for class  2
generating search for class  2
Created folder for class 2
Go to table 2
loading first entry for class 2
Waiting for protection, limitation in class: 2
Waiting for protection, limitation in class: 2


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.86)
Stacktrace:
#0 0x64a169afd8a3 <unknown>
#1 0x64a1697f38c6 <unknown>
#2 0x64a1697ca98d <unknown>
#3 0x64a169868a2d <unknown>
#4 0x64a16987e5a6 <unknown>
#5 0x64a169860343 <unknown>
#6 0x64a169831593 <unknown>
#7 0x64a169831f5e <unknown>
#8 0x64a169ac188b <unknown>
#9 0x64a169ac57e5 <unknown>
#10 0x64a169aaf5b1 <unknown>
#11 0x64a169ac6372 <unknown>
#12 0x64a169a948bf <unknown>
#13 0x64a169aec768 <unknown>
#14 0x64a169aec93b <unknown>
#15 0x64a169afc9f4 <unknown>
#16 0x7ba82c694ac3 <unknown>


In [10]:
import concurrent.futures

def classSearcher(number):
    botClass(str(number))

veces_totales = 45
veces_en_paralelo = 5

with concurrent.futures.ThreadPoolExecutor(max_workers=veces_en_paralelo) as executor:
    for i in range(1, veces_totales + 1, veces_en_paralelo):
        numeros = range(i, min(i + veces_en_paralelo, veces_totales + 1))
        futures = [executor.submit(classSearcher, num) for num in numeros]
        concurrent.futures.wait(futures)





Open Search for class  1
Open Search for class  2
Open Search for class  3
Open Search for class  4
Open Search for class  5
Create folder for class 5
Changing page in class 5
Create folder for class 3
Create folder for class 2
Create folder for class 1
Create folder for class 4
Changing page in class 3
Changing page in class 2
Changing page in class 1
Changing page in class 4
loading first entry for class 4
loading first entry for class 2
loading first entry for class 1
loading first entry for class 3
loading first entry for class 5
I finished i page for class 4
I finished i page for class 2
I finished i page for class 1
I finished i page for class 3
I finished i page for class 5
I finished i page for class 2
I finished i page for class 4
I finished i page for class 1
I finished i page for class 3
I finished i page for class 5
I finished i page for class 4
I finished i page for class 2
I finished i page for class 1
I finished i page for class 3
I finished i page for class 5
I finished

KeyboardInterrupt: 

In [ ]:
    #time.sleep(1)
    #date_before = form.find_element(By.ID, "IngresoDesde");
    #date_after = form.find_element(By.ID, "IngresoHasta");
    #date_before.send_keys("01/01/2020")
    #date_after.send_keys("01/01/2113")

    #pageSize = driver.find_element(By.CLASS_NAME, "page-size")
    #pageSize.click()
    #time.sleep(1)

    #pageList = driver.find_element(By.CLASS_NAME, "page-list")
    #btnGroup = pageList.find_element(By.CLASS_NAME, "btn-group")
    #DropdownMenu = btnGroup.find_elements(By.CLASS_NAME, "dropdown-menu")

    #for optionMenu in DropdownMenu:
    #    if "200" in str(optionMenu.text):
    #        optionMenu.click()
    #time.sleep(11)